In [2]:
import pandas as pd
import time
import openai
openai.api_key = "XXXXXX"

# Guess tweet

In [128]:

def guess_tweet(tweet,model,temperature, instruction):
    print(f"Guessing tweet: '{tweet}'...")

    response = None
    tries = 0
    failed = True
    while(failed):
        try:
            response = openai.ChatCompletion.create(
                model = model,
                temperature=temperature,
                messages=[
                        {"role": "system", "content": f"{instruction}"},
                        {"role": "user", "content": f"'{tweet}'"}
                    ]
            )
            failed = False
            
        except Exception as e:
            print("Caught exception. Waiting...")
            print(e)
            failed = True
            tries += 1
            time.sleep(10)
            
            if tries > 50:
                print("Too many failed. Giving up")
                raise e

    result = ''
    for choice in response.choices:
        result += choice.message.content
        
    return(result)

def process_country(instruction, file, temperature, column, nr_runs, model='gpt-4'):

    sample = pd.read_pickle(file)
    errorcount = 0
    while(True):

        left = sample.loc[sample[column].map(len)<nr_runs]
        print(f"There are {len(left)} left to process.")

        if len(left)==0:
            print("All done!")
            break
        line = left.sample()
        index = line.index.values[0]

        time.sleep(1)
        try:
            guess = guess_tweet(line['text'].values[0],model=model,temperature=temperature, instruction=instruction)
            sample[column][index].append(guess)

            print(f"Guess is: {guess}")
            sample.to_pickle(file)

        except Exception as err:
            print(f"Error. Unexpected {err=}, {type(err)=}")        
            if errorcount < 10:
                errorcount+=1
                print("Error runnning. Will just keep trying though.")
                continue
            else:
                print("Too many errors. Giving up..")
                raise





In [145]:
# Example: The run for germany
COLUMN = 'gpt4_temp02'
INSTRUCTION = '''You will be given a Twitter post from a parliamentarian in Germany, sent during the two months preceding the 2021 German general election, that is, between July 26th, 2021, and September 26th, 2021. Your task is to use your knowledge of Germn politics to make an educated guess on whether the poster belongs to the CDU (Christlich Demokratische Union Deutschlands) or SPD (Sozialdemokratische Partei Deutschlands). Your response MUST BE either 'CDU' or 'SPD'. If you cannot make an educated guess on the basis of the message, just guess either 'CDU' or 'SPD'. Do NOT motivate your answer.'''
FILE = 'tweet_process_germany.pkl'
MODEL = 'gpt-4'
TEMP = 0.2
NR_RUNS = 1

gdf = pd.read_csv('GERMANY_sample_tweets.csv')
gdf[COLUMN] = [[] for i in range(len(gdf))]
gdf.to_pickle(FILE)

process_country(INSTRUCTION, FILE, TEMP, COLUMN, NR_RUNS, MODEL)

# Motivate response to particular questions

In [ ]:
message = '''"Down Syndrome Awareness Month helps raise awareness for what it means to have Down syndrome and how individuals with Down syndrome play a vital role in our lives and communities."'''
answer = 'Republican'

response = openai.ChatCompletion.create(
    model = "gpt-4",
    temperature=0.2,
    messages=[
            {"role": "system", "content": "You will be given a Twitter post from a US politician, sent during the two months preceding the 2020 US presidential election, that is, between September 3rd, 2020, and November 3rd, 2020. Your task is to use your knowledge of US politics to make an educated guess on whether the poster is a Democrat or Republican. Your response MUST BE either 'Democrat' or 'Republican'. You are not allowed to respond anything else. If you cannot make an educated guess on the basis of the message, just guess either 'Democrat' or 'Republican'. Do NOT motivate your answer in your first respponse."},
            {"role": "user", "content": f"'{message}'"},
            {"role": "assistant", "content": f"{answer}"},
            {"role": "user", "content": "Motivate your response."}
        ]
)

result = ''
for choice in response.choices:
    result += choice.message.content

print(result)


# Posts from regular people

In [118]:
def guess_tweet(tweet,model,temperature):
    print(f"Guessing tweet: '{tweet}'...")

    response = None
    tries = 0
    failed = True
    while(failed):
        try:
            response = openai.ChatCompletion.create(
                model = model,
                temperature=temperature,
                messages=[
                        {"role": "system", "content": "You will be given a Twitter post from an individual in the United States, sent during the two months preceding the 2020 US presidential election, that is, between September 3rd, 2020, and November 3rd, 2020. Your task is to use your knowledge of US politics to make an educated guess on whether the poster is a Democrat or Republican. Your response MUST BE either 'Democrat' or 'Republican'. If you cannot make an educated guess on the basis of the message, just guess either 'Democrat' or 'Republican'. Do NOT motivate your answer."},
                        {"role": "user", "content": f"'{tweet}'"}
                    ]
            )
            failed = False
            
        except Exception as e:
            print("Caught exception. Waiting...")
            print(e)
            failed = True
            tries += 1
            time.sleep(10)
            
            if tries > 50:
                print("Too many failed. Giving up")
                raise e

    result = ''
    for choice in response.choices:
        result += choice.message.content
        
    return(result)

COLUMN = 'gpt4_temp02'
sample = pd.read_pickle("regularpeoplellm.pkl")
errorcount = 0

while(True):

    left = sample.loc[sample[COLUMN].map(len)<1]
    print(f"There are {len(left)} left to process.")
    
    if len(left)==0:
        print("All done!")
        break
    line = left.sample()
    index = line.index.values[0]
    
    time.sleep(1)
    try:
        guess = guess_tweet(line['text'].values[0],model='gpt-4',temperature=0.2)
        sample[COLUMN][index].append(guess)

        print(f"Guess is: {guess}")
        sample.to_pickle("randompeople.pkl")

    except Exception as err:
        print(f"Error. Unexpected {err=}, {type(err)=}")        
        if errorcount < 10:
            errorcount+=1
            print("Error runnning. Will just keep trying though.")
            continue
        else:
            print("Too many errors. Giving up.")
            raise
    
    


# Variations of the classification instruction

In [281]:
variation1 = "You will be given a tweet sent by a Senator in the United States. Your job is to say whether you think the Senator belongs to the Democratic or Republican party. The tweet was sent during the two months preceding the 2020 election in the United States. Respond ONLY 'Democrat' or 'Republican'"

FILE = 'US_sample_tweets_llm.pkl'
MODEL = 'gpt-4'
TEMP = 0.2
NR_RUNS = 1
COLUMN = 'gpt4_temp02_variation1'
INSTRUCTION = variation1

gdf = pd.read_pickle(FILE)
gdf[COLUMN] = [[] for i in range(len(gdf))]
gdf.to_pickle(FILE)

process_country(INSTRUCTION, FILE, TEMP, COLUMN, NR_RUNS, MODEL)



In [280]:
variation2 = "Please classify the following Twitter post based on whether the author is a Republican or a Democratic Senator. The tweet was sent before the 2020 election, between September 3rd, 2020, and November 3rd, 2020. Respond ONLY 'Democrat' or 'Republican'"
INSTRUCTION = variation2

FILE = 'US_sample_tweets_llm.pkl'
MODEL = 'gpt-4'
TEMP = 0.2
NR_RUNS = 1
COLUMN = 'gpt4_temp02_variation2'
gdf = pd.read_pickle(FILE)
gdf[COLUMN] = [[] for i in range(len(gdf))]
gdf.to_pickle(FILE)

process_country(INSTRUCTION, FILE, TEMP, COLUMN, NR_RUNS, MODEL)

